In [ ]:
from keras.layers import Input, LSTM, Dense
# from keras.layers import CuDNNLSTM    # an LSTM layer optimized for GPU
# ! accordingly to Stackoverflow cuDNNLSTM are deprecated layers.
from keras.models import Model

import numpy as np

In [ ]:
input_vocab_size = 300
output_vocab_size = 350
num_neurons = 50

encoder_input_data = np.array([])
decoder_input_data = np.array([])
decoder_target_data = np.array([])

batch_size = 32
epochs = 10

## Thought encoder in Keras:

In [ ]:
encoder_inputs = Input(shape=(None, input_vocab_size))
encoder = LSTM(units=num_neurons, return_state=True)
encoder_outputs, shate_h, state_c = encoder(encoder_inputs)
encoder_states = (shate_h, state_c)

## Thought decoder in Keras

In [ ]:
decoder_inputs = Input(shape=(None, output_vocab_size))
decoder_lstm = LSTM(units=num_neurons, return_state=True,
                    return_sequences=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(units=output_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

## Assemble sequence-to-sequence network

In [ ]:
model = Model(inputs=[encoder_inputs, decoder_inputs],
              outputs=decoder_outputs)

## Train a sequence-to-sequence network

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs)

## Generate output sequences

In [ ]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [ ]:
thought_input = [Input(shape=(num_neurons,)), Input(shape=(num_neurons,))]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs,
                                                 initial_state=thought_input)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    inputs=[decoder_inputs] + thought_input,
    outputs=[decoder_outputs] + decoder_states
)

In [ ]:
input_seq = np.array([])
target_seq = np.array([])   # initialized as a start token
stop_condition = False

thought = encoder_model.predict(input_seq)
while not stop_condition:
    output_tokens, h, c = decoder_model.predict([target_seq] + thought)
    # update 'stop_condition'
    # it turns True if either the max num of output sequence tokens is hit or the
    # decoder generate a stop token
